In [ ]:
import json
import lxml.html
import requests
from IPython.display import HTML

USERNAME ="YOUR_USERNAME"
PASSWORD = "YOUR_PASSWORD"
URL_PYBITES_LOGIN = "https://codechalleng.es/login"
URL_PYBITES_BITE = "https://codechalleng.es/bites"

# https://codechalleng.es/bites/api/downloads/bites/213/
# pybites_bite172.zip
# https://codechalleng.es/bites/101/
# pybite_bite101.html

# https://github.com/clamytoe/toepack/blob/master/%7B%7Bcookiecutter.project_name%7D%7D/%7B%7Bcookiecutter.project_name%7D%7D/headers.py
SAFARI_MOBILE = {
    "Accept": "text/html.application/xhtml+xml, application/xml;q=0.9, "
              "*/*;q=0.8",
    "Accept-Language": "en-us",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 11_4 like Mac OS X) "
                  "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.0 "
                  "Mobile/15E148 Safari/604.1",
}

def _get_csrf(content):
    login_html = lxml.html.fromstring(content)
    hiddens_input = login_html.xpath(r'//form/input[@type="hidden"]')
    form = { x.attrib["name"]: x.attrib["value"]
            for x in hiddens_input}
    return form.get("csrfmiddlewaretoken")


def download_resource(bite_num):
    payload = {"username": USERNAME,
               "password": PASSWORD}
    csrfmiddlewaretoken = ""
    url_pybite=f"{URL_PYBITES_BITE}/{bite_num}/"
    
    with requests.Session() as session:
        session.headers.update(SAFARI_MOBILE)
        page_login = session.get(URL_PYBITES_LOGIN)
        
        #csrfmiddlewaretoken = _get_csrf(page_login.text)
        csrfmiddlewaretoken = session.cookies["csrftoken"]
        
        payload.update({"csrfmiddlewaretoken": csrfmiddlewaretoken,
                        "next": "/"})
        session.headers.update({"Referer": URL_PYBITES_LOGIN})
        
        response = session.post(URL_PYBITES_LOGIN,
                                data=payload)
        response = session.get(url_pybite)
        
        bite_html = response.text
        
        return bite_html
        
HTML(download_resource(101))
